# Rekurentne neuronske mreže

U ovoj svesci sumirani su osnovni koncepti vezani za rekturentne neuronske mreže i podrška Keras biblioteke u radu sa njima.

<img src='assets/RNN.png'>
<p style='text-align: right; color: gray; font-size: 10px;'> Slika je pozajmljena iz prezentacije Recurrent Neural Networks Momčila Vasiljevića, MDCS </p>

Ono što je zajedničko za sve rekurentne neuronske mreže je da obrađuju sekvence podataka element po element. Na primer, mogu se obrađivati reči teksta sa ciljem da se predvidi naredna reč ili blok reči (takozvani zadatak dopune teksta, engl. text completition), cene akcija na berzi sa ciljem da se predvidi naredna vrednost i slično. Zato se o rekurentnim neuronskim mrežama obično priča sa aspekta vremenske dimenzije: u obradi u trenutku $t$ uzima se u obzir ulaz mreže $x_t$ i stanje mreže $s_{t-1}$ kojim se sumira ono što je mreža do tog trenutka obradila kako bi se generisao izlaz $o_t$. 

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy as np
np.random.seed(7)

## Jednostavna rekurentna neuronska mreža

Jednostavna rekurentna neuronska mreža je obična potpuno povezana mreža koja u trenutku $t$ na osnovu ulaza $x_t$ i prethodnog izlaza $o_{t-1}$ odlučuje o novom izlazu $o_t$. <img src='assets/simple_RNN.png'>

Sledećim kodom opisan je jedan prolaz kroz ovako definisanu rekurentnu neuronsku mrežu.

Dužinu sekvence ćemo zadati promenljivom `timestamps`.

In [3]:
timestamps = 100

Ulazi u mrežu će nam biti vektori dužine `inuput_size`.  Dužina ovog vektora odgovara broju atributa (engl. features)  koje imamo na nivou pojedinačnih instanci. 

In [4]:
input_size = 32 

Matrica svih ulaza će biti dimenzija `timestamps` x `input_size`. Inicijalizovaćemo je nasumičnim vrednostima.
<img src='assets/sequence_data.png'>

In [5]:
inputs = np.random.random((timestamps, input_size))

Izlazi mreže će biti vektori dužine `output_size`. Sa `output_t_prev` ćemo obeležavati vrednost izlaza za trenutak `t-1`, a sa `output_t` vrednost izlaza za trenutak `t`. Na početku će vektor prethodnih izlaza biti vektor nula.

In [6]:
output_size = 64 

In [7]:
output_t_prev = np.zeros(output_size)

Ovu rekurentnu neuronsku mrežu karakteriše matrica `U` koji povezuje trenutni ulaz `x_t` sa trenutnim izlazom `o_t` i matrica `V` koja povezuje trenutni izlaz `o_t` sa prethodnim izlazom `o_t_prev`. Ove matrice ćemo nasumično generisati. Generisaćemo i nasumični vektor slobodnih članova `b`.

In [8]:
U = np.random.random((output_size, input_size))
V = np.random.random((output_size, output_size))

b = np.random.random(output_size)

<img src='assets/simple_RNN_unrolled.png'>

Jedan prolaz kroz mrežu u obradi sekvence `inputs` je opisan kodom ispod. 

In [9]:
# izlazi mreze
successive_outputs = []

# za svaki ulaz u sekvenci
for input_t in inputs:
    
    # kombinuje se tekuci ulaz input_t i prethodni izlaz output_t_prev i izračunava se novi izlaz
    output_t = np.tanh(np.dot(U, input_t) + np.dot(V, output_t_prev) + b)
    
    # trenutni izlaz mreze treba uzeti u obzir prilikom obrade sledeceg ulaza 
    output_t_prev = output_t.copy()    
    
    # cuvamo izlaz mreze
    successive_outputs.append(output_t)


Niz svih izlaza je dužine koja odgovara dužini sekvence, preciznije oblika `input_size` x `output_size`.

In [10]:
len(successive_outputs)

100

In [11]:
np.array(successive_outputs).shape

(100, 64)

### Odgovarajuća Keras podrška

Rad sa rekurentnim neuronskim mrežama ovog tipa je moguć kroz `SimpleRNN` sloj. Ulaz u mrežu je oblika `(batch_size, timestamps, input_size)` gde `batch_size` predstavlja veličinu paketića tj. broj sekvenci koje mreža očekuje na ulazu, a izlaz može da bude ili u formi `(batch_size, timestamps, output_size)` ili u formi `(batch_size, output_size)`. Ovo ponašanje se kontroliše kroz `return_sequences` argument sloja i u prvom slučaju naglašava da treba sačuvati i vratiti izlaz za svaki ulaz svake sekvence, dok u drugom slučaju naglašava da treba vratiti izlaz samo poslednjeg ulaza sekvence.

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

Ovde ćemo prvo redeklarisati vrednosti koje ćemo koristiti kako se ne bi preklapale sa prethodno generisanim rezultatima.

In [13]:
# velicina paketica
batch_size = 8

In [14]:
timestamps = 100
input_size = 32 
output_size = 64 
inputs = np.random.random((batch_size, timestamps, input_size))

`Varijanta 1`:  vraćaju se izlazi svih ulaza sekvenci

`Varijanta 2`: vraća se izlaz samo poslednjeg ulaza sekvenci

Vrednost parametra `return_sequences` je podrazumevano postavljena na `False`. 

## Rekurentna neuronska mreža sa prenosom skrivenog stanja

Kada se govori o rekurentnim neuronskim mrežama ponajčešće se misli na mreže koje imaju arhitekturu koja je prikazana na slici. 

<img src='assets/RNN_unfolded.png'>

Njome se u trenutku $t$ na osnovu ulaza $x_t$ i prethodnog stanja skrivenog sloja $h_{t-1}$ odlučuje o novom izlazu $o_t$.

Prvo ćemo ponoviti deklaracije promenljivih. Njihovo značenje je i u kontekstu ove arhitetkure identično prethodnim.

In [15]:
timestamps = 100
input_size = 32 
inputs = np.random.random((timestamps, input_size))
output_size = 64 

Veličinu skrivenog sloja mreže ćemo zadati dimenzijom `hidden_size`. Sa `h_t_prev` ćemo obeležavati vrednost skrivenog sloja za trenutak `t-1`, a sa `h_t` vrednost skrivenog sloja za trenutak `t`. Na početku treniranja `h_t_prev` će biti vektor nula.

In [16]:
hidden_size = 32

In [17]:
h_t_prev = np.zeros(hidden_size)

Ovu rekurentnu neuronsku mrežu karakteriše matrica `U` sloja koji povezuje trenutni ulaz `x_t` sa trenutnom vrednošću skrivenog sloja `h_t`, matrica `V` koja povezuje stanje skrivenog sloja iz prethodnog trenutka `h_t_prev` i tekuće stanje `h_t`, i matrica `W` sloja kojom se povezuje skriveno stanje `h_t` sa izlazom `o_t`. 

In [18]:
U = np.random.random((hidden_size, input_size))
V = np.random.random((hidden_size, hidden_size))
W = np.random.random((output_size, hidden_size))

# vektori slobodnih clanova 
b1 = np.random.random(hidden_size)
b2 = np.random.random(output_size)

Jedan prolaz kroz mrežu u obradi sekvence `inputs` je opisan kodom ispod. 

In [19]:
# izlazi mreze
successive_outputs = []

# za svaki ulaz u sekvenci
for input_t in inputs:
    
    # kombinuje se tekuci ulaz input_t i stanje skrivenog sloja mreze iz prethodnog koraka h_t_prev 
    # i izračunava se novo stanje skrivenog sloja 
    h_t = np.tanh(np.dot(U, input_t) + np.dot(V, h_t_prev) + b1)
    
    # izracuna se izlaz mreze
    output_t= np.tanh(np.dot(W, h_t) + b2)

    # trenutne vrednosti skrivenog sloja mreze traba uzeti u obzir prilikom obrade sledeceg ulaza 
    h_t_prev = h_t.copy()    
    
    # cuvamo izlaz mreze
    successive_outputs.append(output_t)


Niz svih izlaza je dužine koja odgovara dužini sekvence, preciznije oblika `timestamps` x `output_size`.

In [20]:
len(successive_outputs)

100

In [21]:
np.array(successive_outputs).shape

(100, 64)

Biblioteka Keras nudi mogućnost korišćenja ovog tipa mreža kroz nadgrađene `LSTM` i `GRU` slojeve. Ovi slojevi koriste nešto kompleksnije ćelije (neurone) kao što su `LSTM` ćelije i `GRU` ćelije. Njihovo ponašanje ćemo prvo opisati, a potom i prikazati kroz Keras API.

## LSTM ćelije

Svojom specifičnom strukturom, LSTM (engl. Long-Short Term Memory) ćelije nude rešenje za praćenje dugoročnih zavisnosti na nivou sekvenci koje zbog problema isčezavajućih gradijenata (praktično) nije bilo moguće sa stanradnim RNN ćelijama. 

Uz vrednosti skrivenog sloja ove ćelije imaju svoje interno stanje, ulaznu kapiju (engl. input gate), kapiju za zaboravljanje (engl. forget gate) i izlaznu kapiju (engl. output gate) kojima, redom, mogu da kontrolišu prisutnost informacija iz prethodnog stanja, njihovu selekciju i generisanje vrednosti koje će dalje propagirati. 

<img src='assets/LSTM_cell_2.png'>

U trenutku $t$ na osnovu ulaza $x_t$, vrednosti skrivenog sloja $h_{t-1}$ i stanja $c_{t-1}$ prethodnog koraka obrade, treba odlučiti o novoj vrednosti skrivenog sloja $h_t$, stanja $c_t$ i izlaza $o_t$. 

Kapijom za zaboravljanje $f_t$ se kontroliše koliko informacija iz prethodnog stanja treba dalje propustiti. Na osnovu vrednosti matrica $W_f$ i $U_f$ i vektora slobodnih članova $b_f$ koje je karakterišu, izračunavaju se vrednosti vektora $f_t$ koje su u opsegu [0, 1]. Pokoordinatnim množenjem sa vrednošću $c_{t-1}$ dobijaju se filtrirane informacije stanja. Dalje se, na sličan način, izračunavaju vrednosti ulazne kapije $i_t$ i vrednosti izlazne kapije $o_t$. Ulaznom kapijom se filtrijaju informacije međustanja ćelije $\tilde{c_t}$ koje treba dodati informacijama koje se filtrijaju kapijom zaboravljanja kako bi se dobilo novo stanje $c_t$. Nova vrednost skrivenog stanja se dobija na osnovu vrednosti izlazne kapije $o_t$ i novog stanja ćelije $c_t$. 

<img src='assets/LSTM_math.png'>

Keras biblioteka podržava rad sa LSTM mrežama kroz LSTM sloj. 

In [22]:
from tensorflow.keras.layers import LSTM

Redeklarisaćemo vrednosti korišćenih promenljivih, a potom i kreirati LSTM sloj.

In [23]:
timestamps = 100
input_size = 32 
inputs = np.random.random((timestamps, input_size))
output_size = 64 

Prvi argument LSTM sloja se zove `units` i predstavlja zapravo dimenzionalnost unutrašnjih slojeva LSTM ćelije, tj. dimenziju skrivenog sloja, dimenziju stanja i dimenziju izlaza. 

Kako su u formulama koje smo zapisali matrice $W_f$, $W_i$, $W_o$, $W_c$ istih dimenzija tj. `output_size` x `input_size`, matrice $U_f$, $U_i$, $U_o$, $U_c$ istih dimenzija tj. `output_size` x `output_size` i vektori slobodnih članova $b_f$, $b_i$, $b_o$, $b_c$  istih dimenzija tj. `output_size` x `1` ukupan broj parametara koje mreža treba da nauči je  4 x `output_size` x `input_size` + 4 x `output_size` x `output_size` + 4 x `output_size` x `1`.

Zapisano na ovaj način, izlaz LSTM sloja je poslednja vrednost skrivenog sloja. Ova vrednost agregira sve što je mreža videla i naučila i predstavlja apstraktnu reprezentaciju cele sekvence. 

Opciono, sloju `LSTM` se mogu dodati parametri `return_sequences` i `return_state`. 

Parametrom `return_sequences`se utiče na vraćanje izlaza tj. skrivenih stanja $h_t$ nakon obrade svakog ulaza sekvence $x_t$.

Ovaj parametar je zgodno koristiti kada se radi sa više, nasloženih jedni na druge (engl. stacked), LSTM slojeva kako bi ulazi i u naredne slojeve mogli da budu sekvence ili kada je potrebna obrada za svaki ulaz sekvence npr. takav zadatak je obrada govora.

Parametrom `return_state` utiče na vraćanje stanja $c_t$ nakon obrade svakog ulaza sekvence $x_t$. Korišćenje ovog parametra zahteva i prebacivanje na funkcionalni API zbog većeg broja izlaza.

In [25]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input

Možemo videti da je sada izlaz iz mreže lista čiji elementi, redom, predstavljaju izlaz mreže, poslednju vrednost skrivenog sloja i poslednju vrednost stanja. 

Informacija o stanju ćelije nam može biti od koristi kada radimo sa modelima koji dele parametre kao kod, npr. enkoder-dekoder arhitektura.

Možemo proveriti i kako izgledaju izlazi mreže kada se kombinuju parametri `return_state` i `return_sequences`.

Sada je izlaz lista čiji su elementi, redom, niz svih skrivenih stanja dobijen prilikom obrade sekvence, poslednja vrednost skrivenog sloja i poslednja vrednost stanja. 

## GRU ćelije

<img src='assets/GRU_cell.png'>

GRU (engl. Gated-Recurent Units) ćelije predstavljaju modifikaciju LSTM ćelija objedinjavanjem kapije ulaza i kapije zaboravljanja u jednu kapiju ažuriranja.

Keras biblioteka nudi podršku u radu sa GRU ćelijama kroz `GRU` sloj. Podešavanja ovog sloja su istovetna sa podešavanjima sloja `LSTM`.

U praksi rad sa GRU ćelijama vodi do bržeg treniranja. 

In [37]:
from tensorflow.keras.layers import GRU

In [38]:
timestamps = 100
input_size = 32 
inputs = np.random.random((timestamps, input_size))
output_size = 64 